## COVID-19 World Vaccination Progress


* This notebook explored the COVID-19 world vaccination progress based on different regions and specific on column
   『 total vaccinations per hundred 』. Since they are missing values around dataset, I didn't find any specific date suit for every region. I picked the date based on the countries in each region that has least missing data. It might be inconsistent throughout the region but the date got pick were actually very close, mostly at the end of January or the begining of February, I think it still provide some interesting information.


* About the plots, Each region has plot include six subplots. Plot on the top-left is the map of the region with the countries; top-middle shows the total vaccinations per hundred across the countries; bottom-left displays type of vaccines used (percentage) on the specific date around the globe and it shares the y-axis with bottom-middle which shows the type of vaccines used in different countries.

In [ ]:
from matplotlib.offsetbox import TextArea, DrawingArea, OffsetImage, AnnotationBbox
from matplotlib.collections import PatchCollection
from matplotlib.patches import PathPatch
from mpl_toolkits.basemap import Basemap
from matplotlib.patches import Polygon
from geopy.geocoders import Nominatim
import matplotlib.pyplot as plt
from matplotlib import image
import seaborn as sns
import pandas as pd
import numpy as np


plt.rcParams['font.family'] = "Krungthep"

In [ ]:
data=pd.read_csv("../input/covid-world-vaccination-progress/country_vaccinations.csv")
continents=pd.read_csv("../input/country-mapping-iso-continent-region/continents2.csv")
cases=pd.read_csv("../input/covid19-global-dataset/worldometer_coronavirus_summary_data.csv")

data_region=data.merge(continents[["region", "sub-region"]], left_on=data.country, right_on=continents.name).drop(columns=["key_0"]).rename(columns={"sub-region":"subregion"})

geolocator = Nominatim(user_agent="geoapiExercises")
latitude, longitude, country=[], [], []
for c in data_region.country.unique().tolist():
    location=geolocator.geocode(c)
    longitude.append(location[1][0])
    latitude.append(location[1][1])
    country.append(c)

countries=pd.DataFrame({"country":country, "latitude":latitude, "longitude":longitude})
data_region=data_region.merge(countries, on="country")

cases.loc[:, "country"]=cases["country"].replace(["UK", "Isle Of Man", "USA"], ["United Kingdom", "Isle of Man", "United States"])
cases_df=pd.DataFrame()
for i in data_region.country.unique():
    df=cases[cases["country"]=="{}".format(i)]
    cases_df=cases_df.append(df)

cases_df.reset_index(drop=True, inplace=True)
cases_df.loc[:, "total_cases_per_hundred"]=round(cases_df["total_cases_per_1m_population"]/10000)
cases_df.loc[:, "total_recovered_rate"]=cases_df["total_recovered"]/cases_df["total_confirmed"]

In [ ]:
def map_country_name(df, countries, color):
    for c in countries:
        df_=df.query("country=='{}'".format(c)).reset_index(drop=True)
        ax1.text(df_.latitude[0]-3, df_.longitude[0], df_.country[0], color=color, fontsize=12, fontweight="bold", zorder=10)
        
def map_shape_colored(countries):
    patches, index, country, count=[], [], "", 0
    m.readshapefile("../input/world-countries-shape-3/World_Countries", "world")
    country_lis=[]
    for info, shape in zip(m.world_info, m.world):   
        if info["COUNTRY"] in countries:
            if info["COUNTRY"]!= country:
                index.append(count)
                country=info["COUNTRY"]
            count+=1
            #The shapes, in this case, are of the type Polygon. To create it, the coordinates must be in a numpy array. The second argument sets the polygon to be closed
            patches.append(Polygon(np.array(shape), True)) 
            country_lis.append(info["COUNTRY"])
    index.append(-1)      
    return index, patches

def create_date_df(df, date):
    
    data_region_vaccine_ratio=df.query("date=='{}'".format(date))["vaccines"].value_counts(normalize=True).reset_index()
    vaccine_index = {vaccine:index for index, vaccine in enumerate(data_region_vaccine_ratio["index"])}
    
    return data_region_vaccine_ratio, vaccine_index
    
def create_region_date_df(df, date, colors):
    
    data_region_date=df.query("date=='{}'".format(date)).sort_values(by=["total_vaccinations_per_hundred"], ascending=False).reset_index(drop=True)
#     print(data_region_date)
    sns.barplot(x="country", y="total_vaccinations_per_hundred", data=data_region_date, palette=colors, alpha=0.7, ax=ax2)

    ax2.set_title("Total Vaccinations per Hundred", size=16, loc="left", pad=10, fontweight="bold")
    ax2.set_xlabel("")
    ax2.set_ylabel("")
    
    return data_region_date

def data_region_vaccine_ratio_plot(df, palette):
    #["#c9643c","#dd6e42","#e3a47a","#e8dab2","#e9e2ce","#eaeaea","#d5e0e5","#c0d6df","#88a2ad","#4f6d7a", "#384D56"]
    sns.barplot(x="vaccines", y="index", data=df, palette=palette, alpha=0.7, ax=ax3)
    ax3.set_title("Type of Vaccines Used (Around the World)", size=16, loc="left", pad=10, fontweight="bold")
    ax3.tick_params(axis="y", which="both", length=0, direction="in", pad=-430)
    ax3.set_xlabel("")
    ax3.set_ylabel("")
    ax3.invert_xaxis()
    
def data_region_vaccine_flag_plot(df, vaccine_index):
    sns.scatterplot(x=df["country"], y=[vaccine_index[i] for i in df["vaccines"]], alpha=0.7, ax=ax4)
    ax4.set_title("Type of Vaccines Used", size=16, loc="left", pad=10, fontweight="bold")
    plt.setp(ax4.get_yticklabels(), visible=False)
    ax4.tick_params(axis="y", which="both", length=0)
    ax4.set_xlabel("")


    for x, y in zip(df["country"], [vaccine_index[i] for i in df["vaccines"]]):
        flag = plt.imread("../input/flagsaroundtheworld/{}.png".format(x.lower()))
        imagebox = OffsetImage(flag, zoom=0.065)
        ab = AnnotationBbox(imagebox, (x, y), bboxprops =dict(edgecolor="white"))
        ax4.add_artist(ab)

def recovered_rate(df, order_color):
    df_all=pd.DataFrame()
    for c in df.country.unique():
        df=cases_df.query("country=='{}'".format(c))
        df_all=df_all.append(df)

    cases=df_all.sort_values(by=["total_cases_per_1m_population"], ascending=False).reset_index()
    cases_color = [order_color.get(k) for k in cases.country]
    sns.barplot(x="country", y="total_cases_per_1m_population", data=cases, palette=cases_color, alpha=0.7, ax=ax5)
    ax5.set_title("Total Cases per Million", size=16, loc="left", pad=10, fontweight="bold")
    ax5.set_xlabel("")
    ax5.set_ylabel("")
    
    recovered=df_all.sort_values(by=["total_recovered_rate"], ascending=False).reset_index()
    recovered_color = [order_color.get(k) for k in recovered.country]
    sns.barplot(x="country", y="total_recovered_rate", data=recovered, palette=recovered_color, alpha=0.7, ax=ax6)
    ax6.set_title("Total Recovered Rate", size=16, loc="left", pad=10, fontweight="bold")
    ax6.set_xlabel("")
    ax6.set_ylabel("")
    
 # for i in data_region.date.unique().tolist():
#     print(i)
#     df=data_region.query("date=='{}'".format(i))
#     print("len of df:" +str(len(df)))
#     print("len of missing in date:" +str(df["total_vaccinations"].isna().sum()))
#     print("diff:" +str(len(df) - df["total_vaccinations"].isna().sum())+"\n")   

### Southern Asia

In [ ]:
data_region_sa=data_region[(data_region["subregion"]=="Southern Asia") & (data_region["date"]=="2021-02-01")].reset_index()
data_region_vaccine_ratio, vaccine_index=create_date_df(data_region, "2021-02-01")

fig=plt.figure(figsize=(27, 12))
gs=fig.add_gridspec(2, 3)
gs.update(wspace=0.1, hspace=0.2)
ax1=fig.add_subplot(gs[0, 0])
ax2=fig.add_subplot(gs[0, 1])
ax3=fig.add_subplot(gs[1, 0])
ax4=fig.add_subplot(gs[1, 1], sharey=ax3, sharex=ax2)
ax5=fig.add_subplot(gs[0, 2])
ax6=fig.add_subplot(gs[1, 2])

fig.set_facecolor("#eddcd2")

m = Basemap(llcrnrlat = 0, llcrnrlon = 50, urcrnrlat = 40, urcrnrlon = 115, ax=ax1)
m.drawcoastlines()
m.drawcountries()
m.fillcontinents(color="#ffffff", lake_color = "#a8dadc");
ax1.set_title("Southern Asia Map", size=16, loc="left", pad=10, fontweight="bold")


map_country_name(data_region_sa, data_region_sa.country.unique().tolist(), "#1D3C45")    
index, patches=map_shape_colored(data_region_sa.country.unique().tolist())
for i, color in zip(index, ["#d08c60", "#D2601A", "#FFF1E1", "#f6aa1c"]):
        ax1.add_collection(PatchCollection(patches[i : i+1], facecolor=color, edgecolor='k', linewidths=1, zorder=3))

palette_order=["#D2601A", "#f6aa1c", "#FFF1E1", "#d08c60"]
data_region_date=create_region_date_df(data_region_sa, "2021-02-01", palette_order)

palette=["#c9643c","#dd6e42","#e3a47a","#e8dab2","#e9e2ce","#eaeaea","#d5e0e5","#c0d6df","#88a2ad","#4f6d7a", "#384D56"]
data_region_vaccine_ratio_plot(data_region_vaccine_ratio, palette)

data_region_vaccine_flag_plot(data_region_date, vaccine_index)

order=[tick.get_text() for tick in ax2.get_xticklabels()]
order_color={}
for o, c in zip(order, palette_order):
    order_color["{}".format(o)]=c
recovered_rate(data_region_sa, order_color)

for s in ["top","right","left", "bottom"]:
    for ax in [ax1, ax2, ax3, ax4, ax5, ax6]:
        ax.spines[s].set_visible(False)

### South-Eastern & Eastern Asia

In [ ]:
data_region_sea_ea=data_region[((data_region["subregion"]=="South-eastern Asia") | (data_region["subregion"]=="Eastern Asia")) & (data_region["date"]=="2021-01-27")]
data_region_vaccine_ratio, vaccine_index=create_date_df(data_region, "2021-01-27")

fig=plt.figure(figsize=(27, 12))
gs=fig.add_gridspec(2, 3)
gs.update(wspace=0.1, hspace=0.2)
ax1=fig.add_subplot(gs[0, 0])
ax2=fig.add_subplot(gs[0, 1])
ax3=fig.add_subplot(gs[1, 0])
ax4=fig.add_subplot(gs[1, 1], sharey=ax3, sharex=ax2)
ax5=fig.add_subplot(gs[0, 2])
ax6=fig.add_subplot(gs[1, 2])

fig.set_facecolor("#eddcd2")

m = Basemap(llcrnrlat =-10 , llcrnrlon = 50, urcrnrlat = 55, urcrnrlon = 150,  ax=ax1)
m.drawcoastlines()
m.drawcountries()
m.fillcontinents(color="#ffffff", lake_color = "#a8dadc");
ax1.set_title("South-Eastern & Eastern Asia Map", size=16, loc="left", pad=10, fontweight="bold")


map_country_name(data_region_sea_ea, data_region_sea_ea.country.unique().tolist(), "#283655")    
index, patches=map_shape_colored(data_region_sea_ea.country.unique().tolist())
for i, color in zip(index, ["#ffe1a8", "#e26d5c", "#472d30", "#723d46"]):
    ax1.add_collection(PatchCollection(patches[i : i+1], facecolor=color, linewidths=1, alpha=0.7, zorder=3))
ax1.add_collection(PatchCollection(patches[index[1] : index[2]], facecolor="#e26d5c", linewidths=1, alpha=0.7, zorder=3))
ax1.add_collection(PatchCollection(patches[index[3] : index[4]], facecolor="#f1a782", linewidths=1, alpha=0.7, zorder=5))

palette_order=["#723d46","#e26d5c","#f1a782","#ffe1a8"]
data_region_date=create_region_date_df(data_region_sea_ea, "2021-01-27", palette_order)

palette=["#723d46","#aa5551","#e26d5c","#e67c66","#ea8a6f","#ee9979","#f1a782","#f5b68c","#f8c495","#ffe1a8"]
data_region_vaccine_ratio_plot(data_region_vaccine_ratio, palette)

data_region_vaccine_flag_plot(data_region_date, vaccine_index)

order=[tick.get_text() for tick in ax2.get_xticklabels()]
order_color={}
for o, c in zip(order, palette_order):
    order_color["{}".format(o)]=c
recovered_rate(data_region_sea_ea, order_color)
  
for s in ["top","right","left", "bottom"]:
    for ax in [ax1, ax2, ax3, ax4, ax5, ax6]:
        ax.spines[s].set_visible(False)

### Western Asia

In [ ]:
data_region_wa=data_region[(data_region["subregion"]=="Western Asia") & (data_region["date"]=="2021-01-29")].dropna(subset=["total_vaccinations"]).reset_index(drop=True)
data_region_vaccine_ratio, vaccine_index=create_date_df(data_region, "2021-01-29")

fig=plt.figure(figsize=(27, 12))
gs=fig.add_gridspec(2, 3)
gs.update(wspace=0.1, hspace=0.2)
ax1=fig.add_subplot(gs[0, 0])
ax2=fig.add_subplot(gs[0, 1])
ax3=fig.add_subplot(gs[1, 0])
ax4=fig.add_subplot(gs[1, 1], sharey=ax3, sharex=ax2)
ax5=fig.add_subplot(gs[0, 2])
ax6=fig.add_subplot(gs[1, 2])

fig.set_facecolor("#eddcd2")

m = Basemap(llcrnrlat=10 , llcrnrlon= 15, urcrnrlat = 50, urcrnrlon = 80, ax=ax1)
m.drawcoastlines()
m.drawcountries()
m.fillcontinents(color="#ffffff", lake_color = "#a8dadc");
ax1.set_title("Western Asia Map", size=16, loc="left", pad=10, fontweight="bold")

map_country_name(data_region_wa, data_region_wa.country.unique().tolist(), "#283655")  
index, patches=map_shape_colored(data_region_wa.country.unique().tolist())

for i, color in zip(index, ["#c18c5d","#ce796b","#495867","#ecc8af","#857262","#e7ad99"]):
    ax1.add_collection(PatchCollection(patches[i : i+1], facecolor=color, linewidths=1, zorder=3))
ax1.add_collection(PatchCollection(patches[index[5] : index[6]], facecolor="#e7ad99", linewidths=1, alpha=0.7, zorder=5))

palette_order=["#495867","#857262","#c18c5d","#ce796b","#e7ad99","#ecc8af"]
data_region_date=create_region_date_df(data_region_wa, "2021-01-29", palette_order)

palette=["#495867","#676565","#857262","#a37f60","#c18c5d","#c88364","#ce796b","#db9382","#e7ad99","#ecc8af"]
data_region_vaccine_ratio_plot(data_region_vaccine_ratio, palette=palette)

data_region_vaccine_flag_plot(data_region_date, vaccine_index)

order=[tick.get_text() for tick in ax2.get_xticklabels()]
order_color={}
for o, c in zip(order, palette_order):
    order_color["{}".format(o)]=c
recovered_rate(data_region_wa, order_color)

for s in ["top","right","left", "bottom"]:
    for ax in [ax1, ax2, ax3, ax4, ax5, ax6]:
        ax.spines[s].set_visible(False)

### Northern Europe

In [ ]:
data_region_ne=data_region[(data_region["subregion"]=="Northern Europe") & (data_region["date"]=="2021-02-02")].dropna(subset=["total_vaccinations_per_hundred"]).reset_index(drop=True)
data_region_vaccine_ratio, vaccine_index=create_date_df(data_region, "2021-02-02")

fig=plt.figure(figsize=(27, 12))
gs=fig.add_gridspec(2, 3)
gs.update(wspace=0.1, hspace=0.25)
ax1=fig.add_subplot(gs[0, 0])
ax2=fig.add_subplot(gs[0, 1])
ax3=fig.add_subplot(gs[1, 0])
ax4=fig.add_subplot(gs[1, 1], sharey=ax3, sharex=ax2)
ax5=fig.add_subplot(gs[0, 2])
ax6=fig.add_subplot(gs[1, 2])

fig.set_facecolor("#eddcd2")
m = Basemap(llcrnrlat=40 , llcrnrlon= -30, urcrnrlat = 80, urcrnrlon = 35, ax=ax1)
m.drawcoastlines()
m.drawcountries()
m.fillcontinents(color="#ffffff", lake_color = "#a8dadc");
ax1.set_title("Northern Europe Map", size=16, loc="left", pad=10, fontweight="bold")

country_list=data_region_ne.country.unique().tolist()
country_list.remove("Isle of Man")
map_country_name(data_region_ne, country_list, "#283655")

df=data_region.query("country=='Isle of Man'").reset_index(drop=True)
ax1.text(df.latitude[0]-3, df.longitude[0]-3, df.country[0], color="#283655", fontsize=12, fontweight="bold", zorder=10)
    
index, patches=map_shape_colored(data_region_ne.country.unique().tolist())


for i, color in zip(index, ["#e3a47a","#e9e2ce","#eaeaea", "#88a2ad", "#e8dab2","#c0d6df","#d5e0e5","#dd6e42","#dd6e42"]):
    ax1.add_collection(PatchCollection(patches[i : i+1], facecolor=color, linewidths=1, zorder=3)) 
ax1.add_collection(PatchCollection(patches[index[5] : index[6]], facecolor="#c0d6df", linewidths=1, alpha=0.7, zorder=5))
ax1.add_collection(PatchCollection(patches[index[7] : index[8]], facecolor="#dd6e42", linewidths=1, alpha=0.7, zorder=5))

palette=["#dd6e42","#e0895e","#e3a47a","#e8dab2","#e9e2ce","#eaeaea","#d5e0e5","#c0d6df","#88a2ad"]
data_region_date=create_region_date_df(data_region_ne, "2021-02-02", palette)
ax2.tick_params(axis="x", rotation=20)

data_region_vaccine_ratio_plot(data_region_vaccine_ratio, palette=palette)

data_region_vaccine_flag_plot(data_region_date, vaccine_index)
ax4.tick_params(axis="x", rotation=20)

order=[tick.get_text() for tick in ax2.get_xticklabels()]
order_color={}
for o, c in zip(order, palette):
    order_color["{}".format(o)]=c
recovered_rate(data_region_ne, order_color)
ax5.tick_params(axis="x", rotation=20)
ax6.tick_params(axis="x", rotation=20)

for s in ["top","right","left", "bottom"]:
    for ax in [ax1, ax2, ax3, ax4, ax5, ax6]:
        ax.spines[s].set_visible(False)

### Eastern Europe

In [ ]:
data_region_ee=data_region[(data_region["subregion"]=="Eastern Europe") & (data_region["date"]=="2021-02-02")].reset_index(drop=True)
data_region_vaccine_ratio, vaccine_index=create_date_df(data_region, "2021-02-02")

fig=plt.figure(figsize=(27, 12))
gs=fig.add_gridspec(2, 3)
gs.update(wspace=0.1, hspace=0.25)
ax1=fig.add_subplot(gs[0, 0])
ax2=fig.add_subplot(gs[0, 1])
ax3=fig.add_subplot(gs[1, 0])
ax4=fig.add_subplot(gs[1, 1], sharey=ax3, sharex=ax2)
ax5=fig.add_subplot(gs[0, 2])
ax6=fig.add_subplot(gs[1, 2])

fig.set_facecolor("#eddcd2")

m = Basemap(llcrnrlat=35 , llcrnrlon= 5, urcrnrlat = 60, urcrnrlon = 45, ax=ax1)
m.drawcoastlines()
m.drawcountries()
m.fillcontinents(color="#ffffff", lake_color = "#a8dadc");
ax1.set_title("Eastern Europe Map", size=16, loc="left", pad=10, fontweight="bold")

map_country_name(data_region_ee, data_region_ee.country.unique().tolist(), "#283655") 
index, patches=map_shape_colored(data_region_ee.country.unique().tolist())

for i, color in zip(index,  ["#fe938c","#e6b89c","#ead2ac","#9cafb7","#4281a4"]):
        ax1.add_collection(PatchCollection(patches[i : i+1], facecolor=color, edgecolor='k', linewidths=1, alpha=0.7, zorder=3))
        
palette_order=["#4281a4","#9cafb7","#e6b89c","#ead2ac","#fe938c"]
data_region_date=create_region_date_df(data_region_ee, "2021-02-02", palette_order)

palette=["#4281a4","#6f98ae","#86a4b3","#9cafb7","#c3c1b2","#ead2ac","#e8c5a4","#e6b89c","#f2a694","#fe938c"]
data_region_vaccine_ratio_plot(data_region_vaccine_ratio, palette=palette)

data_region_vaccine_flag_plot(data_region_date, vaccine_index)

order=[tick.get_text() for tick in ax2.get_xticklabels()]
order_color={}
for o, c in zip(order, palette_order):
    order_color["{}".format(o)]=c
recovered_rate(data_region_ee, order_color)

for s in ["top","right","left", "bottom"]:
    for ax in [ax1, ax2, ax3, ax4, ax5, ax6]:
        ax.spines[s].set_visible(False)

### Western Europe

In [ ]:
data_region_we=data_region[(data_region["subregion"]=="Western Europe") & (data_region["date"]=="2021-01-31")].reset_index(drop=True)
data_region_vaccine_ratio, vaccine_index=create_date_df(data_region, "2021-01-31")

fig=plt.figure(figsize=(27, 12))
gs=fig.add_gridspec(2, 3)
gs.update(wspace=0.1, hspace=0.25)
ax1=fig.add_subplot(gs[0, 0])
ax2=fig.add_subplot(gs[0, 1])
ax3=fig.add_subplot(gs[1, 0])
ax4=fig.add_subplot(gs[1, 1], sharey=ax3, sharex=ax2)
ax5=fig.add_subplot(gs[0, 2])
ax6=fig.add_subplot(gs[1, 2])

fig.set_facecolor("#eddcd2")

m = Basemap(llcrnrlat=35 , llcrnrlon= -10, urcrnrlat = 60, urcrnrlon = 30, ax=ax1)
m.drawcoastlines()
m.drawcountries()
m.fillcontinents(color="#ffffff", lake_color = "#a8dadc");
ax1.set_title("Western Europe Map", size=16, loc="left", pad=10, fontweight="bold")

country_list=data_region_we.country.unique().tolist()
country_list.remove("Austria")
df=data_region.query("country=='Austria'").reset_index(drop=True)
ax1.text(df.latitude[0]+1, df.longitude[0], df.country[0], color="#283655", fontsize=12, fontweight="bold", zorder=10)

map_country_name(data_region_we, country_list, "#1D3C45") 
index, patches=map_shape_colored(data_region_we.country.unique().tolist())

for i, color in zip(index,  ["#b2b187","#da6e48","#f4d58d","#cd3a26","#708d81","#668075","#bf0603"]):
         ax1.add_collection(PatchCollection(patches[i : i+1], facecolor=color, edgecolor='k', linewidths=1, alpha=0.7, zorder=3))

palette_order=["#bf0603","#cd3a26","#da6e48","#f4d58d","#b2b187","#708d81","#668075"]
data_region_date=create_region_date_df(data_region_we, "2021-01-31", palette_order)

palette=["#bf0603","#cd3a26","#da6e48","#e7a26b","#f4d58d","#b2b187","#919f84","#708d81","#6b877b","#668075"]
data_region_vaccine_ratio_plot(data_region_vaccine_ratio, palette=palette)

data_region_vaccine_flag_plot(data_region_date, vaccine_index)

order=[tick.get_text() for tick in ax2.get_xticklabels()]
order_color={}
for o, c in zip(order, palette_order):
    order_color["{}".format(o)]=c
recovered_rate(data_region_we, order_color)

for s in ["top","right","left", "bottom"]:
    for ax in [ax1, ax2, ax3, ax4, ax5, ax6]:
        ax.spines[s].set_visible(False)

### Southern Europe

In [ ]:
data_region_se=data_region[(data_region["subregion"]=="Southern Europe") & (data_region["date"]=="2021-02-01")].dropna(subset=["total_vaccinations_per_hundred"]).reset_index(drop=True)
data_region_vaccine_ratio, vaccine_index=create_date_df(data_region, "2021-02-01")

fig=plt.figure(figsize=(27, 12))
gs=fig.add_gridspec(2, 3)
gs.update(wspace=0.1, hspace=0.25)
ax1=fig.add_subplot(gs[0, 0])
ax2=fig.add_subplot(gs[0, 1])
ax3=fig.add_subplot(gs[1, 0])
ax4=fig.add_subplot(gs[1, 1], sharey=ax3, sharex=ax2)
ax5=fig.add_subplot(gs[0, 2])
ax6=fig.add_subplot(gs[1, 2])

fig.set_facecolor("#eddcd2")

m = Basemap(llcrnrlat=27 , llcrnrlon= -15, urcrnrlat = 55, urcrnrlon = 30, ax=ax1)
m.drawcoastlines()
m.drawcountries()
m.fillcontinents(color="#ffffff", lake_color = "#a8dadc");
ax1.set_title("Southern Europe Map", size=16, loc="left", pad=10, fontweight="bold")

country_list=data_region_se.country.unique().tolist()
country_list.remove("Spain")
df=data_region.query("country=='Spain'").reset_index(drop=True)
ax1.text(df.latitude[0]+1, df.longitude[0], df.country[0], color="#283655", fontsize=12, fontweight="bold", zorder=10)

map_country_name(data_region_se, country_list, "#1D3C45")    
index, patches=map_shape_colored(data_region_se.country.unique().tolist())

for i, color in zip(index,  ["#f95738","#ee964b","#f47742","#f4d35e","#41698e","#f1b555","#7a94a5","#ebebd3", "#d68c45"]):
         ax1.add_collection(PatchCollection(patches[i : i+1], facecolor=color, edgecolor='k', linewidths=1, alpha=0.7, zorder=3))

palette_order=["#083d77","#41698e","#7a94a5","#ebebd3","#f4d35e","#f1b555","#ee964b","#f47742","#f95738"]
data_region_date=create_region_date_df(data_region_se, "2021-02-01", palette_order)

palette=["#083d77","#41698e","#7a94a5","#ebebd3","#f4d35e","#f1b555","#ee964b","#f47742","#f95738"]
data_region_vaccine_ratio_plot(data_region_vaccine_ratio, palette=palette)

data_region_vaccine_flag_plot(data_region_date, vaccine_index)

order=[tick.get_text() for tick in ax2.get_xticklabels()]
order_color={}
for o, c in zip(order, palette_order):
    order_color["{}".format(o)]=c
recovered_rate(data_region_se, order_color)

for s in ["top","right","left", "bottom"]:
    for ax in [ax1, ax2, ax3, ax4, ax5, ax6]:
        ax.spines[s].set_visible(False)

### Northern America

In [ ]:
data_region_na=data_region[(data_region["subregion"]=="Northern America") & (data_region["date"]=="2021-02-02")].reset_index(drop=True)
data_region_vaccine_ratio, vaccine_index=create_date_df(data_region, "2021-02-02")

fig=plt.figure(figsize=(27, 12))
gs=fig.add_gridspec(2, 3)
gs.update(wspace=0.1, hspace=0.25)
ax1=fig.add_subplot(gs[0, 0])
ax2=fig.add_subplot(gs[0, 1])
ax3=fig.add_subplot(gs[1, 0])
ax4=fig.add_subplot(gs[1, 1], sharey=ax3, sharex=ax2)
ax5=fig.add_subplot(gs[0, 2])
ax6=fig.add_subplot(gs[1, 2])

fig.set_facecolor("#eddcd2")

m = Basemap(llcrnrlat=0 , llcrnrlon= -180, urcrnrlat = 90, urcrnrlon = -30, ax=ax1)
m.drawcoastlines()
m.drawcountries()
m.fillcontinents(color="#ffffff", lake_color = "#a8dadc");
ax1.set_title("Northern America Map", size=16, loc="left", pad=10, fontweight="bold")

map_country_name(data_region_na, data_region_na.country.unique().tolist(), "#1D3C45")    
index, patches=map_shape_colored(data_region_na.country.unique().tolist())

ax1.add_collection(PatchCollection(patches[index[0] : index[1]], facecolor="#F2A568", linewidths=1, alpha=0.7, zorder=3))
ax1.add_collection(PatchCollection(patches[index[1] : index[2]], facecolor="#707469", linewidths=1, alpha=0.7, zorder=3))

palette_order=["#707469","#F2A568"]
data_region_date=create_region_date_df(data_region_na, "2021-02-02", palette_order)

palette=["#4c5657","#707469","#94927b","#b8b08d","#d5c290","#f2d492","#f2b576","#f2a568","#f29559","#f39f68"]
data_region_vaccine_ratio_plot(data_region_vaccine_ratio, palette=palette)

data_region_vaccine_flag_plot(data_region_date, vaccine_index)

order=[tick.get_text() for tick in ax2.get_xticklabels()]
order_color={}
for o, c in zip(order, palette_order):
    order_color["{}".format(o)]=c
recovered_rate(data_region_na, order_color)

for s in ["top","right","left", "bottom"]:
    for ax in [ax1, ax2, ax3, ax4, ax5, ax6]:
        ax.spines[s].set_visible(False)

### Latin America and the Caribbean

In [ ]:
data_region_lac=data_region[(data_region["subregion"]=="Latin America and the Caribbean") & (data_region["date"]=="2021-01-28")].dropna(subset=["total_vaccinations_per_hundred"]).reset_index(drop=True)
data_region_vaccine_ratio, vaccine_index=create_date_df(data_region, "2021-01-28")

fig=plt.figure(figsize=(27, 12))
gs=fig.add_gridspec(2, 3)
gs.update(wspace=0.1, hspace=0.25)
ax1=fig.add_subplot(gs[0, 0])
ax2=fig.add_subplot(gs[0, 1])
ax3=fig.add_subplot(gs[1, 0])
ax4=fig.add_subplot(gs[1, 1], sharey=ax3, sharex=ax2)
ax5=fig.add_subplot(gs[0, 2])
ax6=fig.add_subplot(gs[1, 2])

fig.set_facecolor("#eddcd2")

m = Basemap(llcrnrlat=-65 , llcrnrlon= -160, urcrnrlat = 35, urcrnrlon = 5, ax=ax1)
m.drawcoastlines()
m.drawcountries()
m.fillcontinents(color="#ffffff", lake_color = "#a8dadc");
ax1.set_title("Latin America and the Caribbean Map", size=16, loc="left", pad=10, fontweight="bold")


map_country_name(data_region_lac, data_region_lac.country.unique().tolist(), "#1D3C45")    
index, patches=map_shape_colored(data_region_lac.country.unique().tolist())

for i, color in zip(index,  ["#89bd9e","#e26f65","#a3c099","#eba47f","#e57f76", "#f0c987"]):
    ax1.add_collection(PatchCollection(patches[i : i+1], facecolor=color, edgecolor='k', linewidths=1, alpha=0.7, zorder=3))

palette_order=["#89bd9e","#a3c099","#f0c987","#eba47f","#e57f76","#e26f65"]
data_region_date=create_region_date_df(data_region_lac, "2021-01-28", palette_order)

palette=["#688e77","#729c83","#7dac90","#89bd9e","#a3c099","#f0c987","#eba47f","#e57f76","#e26f65","#df5e53"]
data_region_vaccine_ratio_plot(data_region_vaccine_ratio, palette=palette)

data_region_vaccine_flag_plot(data_region_date, vaccine_index)

order=[tick.get_text() for tick in ax2.get_xticklabels()]
order_color={}
for o, c in zip(order, palette_order):
    order_color["{}".format(o)]=c
recovered_rate(data_region_lac, order_color)

for s in ["top","right","left", "bottom"]:
    for ax in [ax1, ax2, ax3, ax4, ax5, ax6]:
        ax.spines[s].set_visible(False)

### Northern Africa

In [ ]:
data_region_nf=data_region[(data_region["subregion"]=="Northern Africa") & (data_region["date"]=="2021-01-30")].dropna(subset=["total_vaccinations_per_hundred"]).reset_index(drop=True)
data_region_vaccine_ratio, vaccine_index=create_date_df(data_region, "2021-01-30")

fig=plt.figure(figsize=(27, 12))
gs=fig.add_gridspec(2, 3)
gs.update(wspace=0.1, hspace=0.25)
ax1=fig.add_subplot(gs[0, 0])
ax2=fig.add_subplot(gs[0, 1])
ax3=fig.add_subplot(gs[1, 0])
ax4=fig.add_subplot(gs[1, 1], sharey=ax3, sharex=ax2)
ax5=fig.add_subplot(gs[0, 2])
ax6=fig.add_subplot(gs[1, 2])

fig.set_facecolor("#eddcd2")

m = Basemap(llcrnrlat=0 , llcrnrlon= -25, urcrnrlat = 50, urcrnrlon = 60, ax=ax1)
m.drawcoastlines()
m.drawcountries()
m.fillcontinents(color="#ffffff", lake_color = "#a8dadc");
ax1.set_title("Northern Africa Map", size=16, loc="left", pad=10, fontweight="bold")

map_country_name(data_region_nf, data_region_nf.country.unique().tolist(), "#1D3C45")    
index, patches=map_shape_colored(data_region_nf.country.unique().tolist())

for i, color in zip(index,  ["#ffefd3","#ffc49b","#adb6c4"]):
    ax1.add_collection(PatchCollection(patches[i : i+1], facecolor=color, edgecolor='k', linewidths=1, alpha=0.7, zorder=3))

palette_order=["#adb6c4","#ffefd3","#ffc49b"]
data_region_date=create_region_date_df(data_region_nf, "2021-01-30", palette_order)

palette=["#294c60","#4a6779","#6b8192","#adb6c4","#d6d3cc","#ffefd3","#ffdab7","#ffc49b","#ffb685","#ffa970"]
data_region_vaccine_ratio_plot(data_region_vaccine_ratio, palette=palette)

data_region_vaccine_flag_plot(data_region_date, vaccine_index)

order=[tick.get_text() for tick in ax2.get_xticklabels()]
order_color={}
for o, c in zip(order, palette_order):
    order_color["{}".format(o)]=c
recovered_rate(data_region_nf, order_color)

for s in ["top","right","left", "bottom"]:
    for ax in [ax1, ax2, ax3, ax4, ax5, ax6]:
        ax.spines[s].set_visible(False)

#### Interesting Things Found

* Pupular vaccines used around the world are **Pfizer/BioNTech** and **Moderna**.
* Type of vaccines used in Europe are pretty consistent. Most of them are Pfizer/BioNTech and Moderna compare to other regions which are quite diverse.




* Countries in Western Asia generally have higher total vaccinations ratio especially in Israel and United Arab Emirates compare to other Asia countries.
* In Northern Europe, United Kingdom has highest total vaccinations ratio and quite a gap to others.
* Northern Africa has lowest total vaccinations ratio after Southern Asia.


Thanks for reading my notebook, if you have any suggestion please feel free to leave comments.
This notebook is still in progress, I will update soon!